In [1]:
import os
import sys

# Add the repository root to the Python path
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, repo_root)

# Set the working directory to the repository root
os.chdir(repo_root)

In [2]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva.core.ermrest_model import (
    builtin_types, 
    Schema, 
    Table, 
    Column, 
    Key, 
    ForeignKey
)
from deriva_ml import DatasetSpec, ExecutionConfiguration, DerivaML, Workflow
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'dev.eye-ai.org'
# host = 'www.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

You are already logged in.


In [10]:
cache_dir = '/Users/vivi/Desktop/eye_ai/execution_cache'
working_dir = '/Users/vivi/Desktop/eye_ai/execution_cache'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [5]:
workflow_instance = Workflow(
    name="Create Condition_Label feature",
    workflow_type="Feature_Creation",
    url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/feature/create_chart_label.ipynb',
    is_notebook=True
)

config = ExecutionConfiguration(
    workflow= workflow_instance, # dev'5-SG9W'
    description="Create Condition_Label for multimodal data")

# Initialize execution
execution = EA.create_execution(config)

Execution RID: https://dev.eye-ai.org/id/eye-ai/6-9SHY@33N-3AG0-31XY

2025-07-22 16:54:27,647 - INFO - Downloading assets ...
2025-07-22 16:54:28,104 - INFO - Initialize status finished.


In [6]:
print(execution)

caching_dir: /Users/vivi/Desktop/eye_ai/execution_cache
_working_dir: /Users/vivi/Desktop/eye_ai/execution_cache/vivi/EyeAI_working
execution_rid: 6-9SHY
workflow_rid: 6-98GP
asset_paths: {}
configuration: datasets=[] assets=[] workflow=Workflow(name='Create Condition_Label feature', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/feature/create_chart_label.ipynb', workflow_type='Feature_Creation', version=None, description=None, rid=None, checksum=None, is_notebook=True) parameters={} description='Create Condition_Label for multimodal data' argv=['/opt/anaconda3/envs/eye_ai/lib/python3.12/site-packages/ipykernel_launcher.py', '--f=/Users/vivi/Library/Jupyter/runtime/kernel-v30d353d52283c34835b78149d84820243b9c560d9.json']


# Create Feature

In [7]:
image_side_table = EA.model.schemas['eye-ai'].tables['Image_Side']
condition_label_table = EA.model.schemas['eye-ai'].tables['Condition_Label']
severity_label_table = EA.model.schemas['eye-ai'].tables['Severity_Label']


In [11]:
# severity_cv = EA.create_vocabulary(vocab_name='Chart_Label', schema='eye-ai')
chart_label_provider = Column.define(
    'Chart_Label_Provider',
    builtin_types.text,
    nullok=True
    )

severity_feature = EA.create_feature(target_table='Subject', feature_name='Chart_Label',
                                     terms=[condition_label_table, severity_label_table, image_side_table],
                                     metadata=[chart_label_provider])


# Compute Feature
## Retrived data

In [11]:
chart_label = pd.read_excel('/Users/vivi/Desktop/eye_ai/data_collection/chart_diagnosis_output-05-28-2025_NP.xlsx')
chart_label['chart_label_provider'] = chart_label['chart_label_provider'].astype(str)

chart_label['chart_label'] = chart_label['chart_label'].replace('Other Glaucoma', 'Other')
chart_label['severity_chart_label'] = chart_label['severity_chart_label'].replace('Indeterminate', 'Unspecified/Indeterminate')
chart_label['severity_chart_label'] = chart_label['severity_chart_label'].replace('Suspect', 'GS')
# chart_label

# Feature ingestion

In [12]:
feature_name = 'Chart_Label'
Feature = EA.feature_record_class('Subject', feature_name)
Feature

deriva_ml.feature.SubjectFeatureChart_Label

In [13]:
from IPython.display import Markdown, display
display(
    Markdown('### Feature Name'),
    [ f'Name: {c.name}, Required: {not c.nullok}' for c in Feature.feature.feature_columns]
)

### Feature Name

['Name: Condition_Label, Required: True',
 'Name: Image_Side, Required: True',
 'Name: Chart_Label_Provider, Required: False',
 'Name: Severity_Label, Required: True']

In [54]:
chart_label_feature_list = [Feature(
    Execution=execution.execution_rid,
    Subject=row['RID_Subject'],
    Chart_Label_Provider=row['chart_label_provider'],
    Image_Side=row['Side'],
    Condition_Label=row['chart_label'],
    Severity_Label=row['severity_chart_label'],
    Feature_Name=feature_name,
    ) for index, row in chart_label.iterrows()]

In [56]:
# len(chart_label_feature_list)
chart_label_feature_list

[SubjectFeatureChart_Label(Execution='6-9S46', Feature_Name='Chart_Label', Condition_Label='POAG', Image_Side='Right', Chart_Label_Provider='Benjamin Xu', Severity_Label='Unspecified/Indeterminate', Subject='2-7P00'),
 SubjectFeatureChart_Label(Execution='6-9S46', Feature_Name='Chart_Label', Condition_Label='POAG', Image_Side='Left', Chart_Label_Provider='Benjamin Xu', Severity_Label='Unspecified/Indeterminate', Subject='2-7P00'),
 SubjectFeatureChart_Label(Execution='6-9S46', Feature_Name='Chart_Label', Condition_Label='POAG', Image_Side='Right', Chart_Label_Provider='Benjamin Xu', Severity_Label='Unspecified/Indeterminate', Subject='2-7MTC'),
 SubjectFeatureChart_Label(Execution='6-9S46', Feature_Name='Chart_Label', Condition_Label='POAG', Image_Side='Left', Chart_Label_Provider='Benjamin Xu', Severity_Label='Unspecified/Indeterminate', Subject='2-7MTC'),
 SubjectFeatureChart_Label(Execution='6-9S46', Feature_Name='Chart_Label', Condition_Label='POAG', Image_Side='Right', Chart_Label

In [57]:
execution.add_features(chart_label_feature_list)

In [64]:
file = EA.working_dir/'deriva-ml/execution/6-9S46/feature/eye-ai/Subject/Chart_Label/Chart_Label.jsonl'

dict_list = [json.loads(line) for line in open(file, 'r')]
dict_list

[{'Execution': '6-9S46',
  'Feature_Name': 'Chart_Label',
  'Condition_Label': 'POAG',
  'Image_Side': 'Right',
  'Chart_Label_Provider': 'Benjamin Xu',
  'Severity_Label': 'Unspecified/Indeterminate',
  'Subject': '2-7P00'},
 {'Execution': '6-9S46',
  'Feature_Name': 'Chart_Label',
  'Condition_Label': 'POAG',
  'Image_Side': 'Left',
  'Chart_Label_Provider': 'Benjamin Xu',
  'Severity_Label': 'Unspecified/Indeterminate',
  'Subject': '2-7P00'},
 {'Execution': '6-9S46',
  'Feature_Name': 'Chart_Label',
  'Condition_Label': 'POAG',
  'Image_Side': 'Right',
  'Chart_Label_Provider': 'Benjamin Xu',
  'Severity_Label': 'Unspecified/Indeterminate',
  'Subject': '2-7MTC'},
 {'Execution': '6-9S46',
  'Feature_Name': 'Chart_Label',
  'Condition_Label': 'POAG',
  'Image_Side': 'Left',
  'Chart_Label_Provider': 'Benjamin Xu',
  'Severity_Label': 'Unspecified/Indeterminate',
  'Subject': '2-7MTC'},
 {'Execution': '6-9S46',
  'Feature_Name': 'Chart_Label',
  'Condition_Label': 'POAG',
  'Image_Si

In [66]:
feature_table = EA.catalog.schemas['eye-ai'].tables['Execution_Subject_Chart_Label']

feature_table.insert(dict_list)

AttributeError: 'ErmrestCatalog' object has no attribute 'schemas'

In [58]:
execution.upload_execution_outputs()

2025-07-22 16:10:24,068 - INFO - Uploading execution files...
2025-07-22 16:10:24,668 - INFO - Initializing uploader: GenericUploader v1.7.10 [Python 3.12.9, macOS-15.5-arm64-arm-64bit]
2025-07-22 16:10:24,785 - INFO - Attempting to acquire a dependent lock in [/Users/vivi/Desktop/eye_ai/execution_cache/vivi/EyeAI_working/deriva-ml/execution/6-9S46/asset]
2025-07-22 16:10:24,787 - INFO - Scanning files in directory [/Users/vivi/Desktop/eye_ai/execution_cache/vivi/EyeAI_working/deriva-ml/execution/6-9S46/asset]...
2025-07-22 16:10:24,787 - INFO - Including file: [/Users/vivi/Desktop/eye_ai/execution_cache/vivi/EyeAI_working/deriva-ml/execution/6-9S46/asset/deriva-ml/Execution_Metadata/configuration.json].
2025-07-22 16:10:24,788 - INFO - Including file: [/Users/vivi/Desktop/eye_ai/execution_cache/vivi/EyeAI_working/deriva-ml/execution/6-9S46/asset/deriva-ml/Execution_Metadata/environment_snapshot_20250722_154440.txt].
2025-07-22 16:10:24,788 - INFO - Processing: [/Users/vivi/Desktop/eye

DerivaMLException: Feature Subject:Chart_Label doesn't exist.

In [51]:
EA.feature_record_class('Subject', feature_name)

deriva_ml.feature.SubjectFeatureChart_Label

In [52]:
EA.find_features('Image')

[Feature(target_table=Image, feature_name=Fundus_Laterality, feature_table=Execution_Image_Fundus_Laterality),
 Feature(target_table=Image, feature_name=Fundus_Angle, feature_table=Execution_Image_Fundus_Angle),
 Feature(target_table=Image, feature_name=Annotation, feature_table=Annotation),
 Feature(target_table=Image, feature_name=Image_Diagnosis, feature_table=Image_Diagnosis)]

In [53]:
EA.find_features('Subject')
# EA.lookup_feature('Image', 'Fundus_Laterality')

[Feature(target_table=Subject, feature_name=test_feature, feature_table=Execution_Subject_test_feature),
 Feature(target_table=Subject, feature_name=Chart_Label, feature_table=Execution_Subject_Chart_Label)]

In [59]:
table = EA.model.schemas['eye-ai'].tables['Subject']

In [60]:
for t in table.find_associations():
    print(t.name)

Subject_Dataset
Subject_image_quality_factor_Subject


In [35]:
# test
test_feature = EA.create_feature(target_table='Subject', feature_name='test_feature',
                                terms=[condition_label_table],
                                metadata=[chart_label_provider])


In [37]:
newf = EA.feature_record_class('Subject', 'test_feature')
newf

deriva_ml.feature.SubjectFeaturetest_feature

In [43]:
newt = EA.model.schemas['eye-ai'].tables['Execution_Subject_test_feature']
newt
lookup_feature = EA.lookup_feature('Subject', 'test_feature')
lookup_feature

Feature(target_table=Subject, feature_name=test_feature, feature_table=Execution_Subject_test_feature)